In [1]:
%%html
<style>
table {float:left}
</style>

In [2]:
# Cross streets, [lat min, lat max], [lon min, lon max]
streets = {
    "El Molino": [
        {"Name": "Bonita Dr to Atchison St",
         "lat min": 0, "lat max": 90,
         "lon min": -118.1373, "lon max": -118.1367}],
    "Wilson": [
        {"Name": "California Blvd to Washington Blvd",
         "lat min": 34.13578, "lat max": 34.17,
         "lon min": -118.12845, "lon max": -118.1276}],
    "Sierra Bonita": [
        {"Name": "Colorado Blvd to Walnut Ave",
         "lat min": 34.14599, "lat max": 34.1463,
         "lon min": -118.1172, "lon max": -118.117},
        {"Name": "Walnut Ave to Villa St",
         "lat min": 34.14956, "lat max": 34.15445,
         "lon min": -118.1173, "lon max": -118.1169},
        {"Name": "Villa St to Washington Blvd",
         "lat min": 34.1544, "lat max": 34.16921,
         "lon min": -118.1179, "lon max": -118.1176}],
    "Craig": [
        {"Name": "Del Mar Blvd to Orange Grove Blvd",
         "lat min": 34.142, "lat max": 34.1595,
         "lon min": -118.1064, "lon max": -118.106}]
}

# Greenways Traffic Collision Stats
This analysis uses the Pasadena Traffic Collision stats, which Pasadena provides at https://data.cityofpasadena.net/datasets/85f49ea583c24056968bee6e28162da4/explore. Pasadena last updated the data 5/5/22 and the data set was downloaded 7/12/23.

This analysis uses GPS coordinates to assign the data to streets.

| Street | Section | Latitude | Longitude |
| :- | :- | :-: | :-: |
| El Molino | Bonita Dr to Atchison St | - | -118,1373, -118.1367 |
| Wilson | California Blvd to Washington Blvd | >34.13578 | -118.12845, -118.1276 |
| Sierra Bonita | Colorado Blvd to Walnut Ave | 34.14599, 34.1463 |  -118.1172, -118.117 |
|| Walnut Ave to Villa St | 34.14956, 34.15445 | -118.1173, -118.1169 |
|| Villa St to Washington Blvd | 34.1544, 34.16921 | -118.1179, -118.1176 |
| Craig | Del Mar Blvd to Orange Grove Blvd | 34.14607, 34.1595 | -118.1284, -118.1278 |

## Import libraries and Data

In [4]:
import pandas as pd
import plotly.express as px
from IPython.display import display, Markdown
df = pd.read_pickle('Traffic_Collisions.pkl')
df.loc[(df.PartyAge1 <= 18),  'AgeGroup1'] = 'Child'
df.loc[(df.PartyAge1 < 65) & (df.PartyAge1 > 18),  'AgeGroup1'] = 'Adult'
df.loc[(df.PartyAge1 >= 65),  'AgeGroup1'] = 'Senior'
df.loc[(df.PartyAge2 <= 18),  'AgeGroup2'] = 'Child'
df.loc[(df.PartyAge2 < 65) & (df.PartyAge2 > 18),  'AgeGroup2'] = 'Adult'
df.loc[(df.PartyAge2 >= 65),  'AgeGroup2'] = 'Senior'



## Longitude Analysis

In [5]:
greenways_df = {}
for street, sections in streets.items():
    greenways_df[street] = pd.DataFrame()
    for section in sections:
        section_df = df.loc[(df['lat'] >= section['lat min']) & 
                            (df['lat'] <= section['lat max']) &
                            (df['lon'] >= section['lon min']) & 
                            (df['lon'] <= section['lon max'])]
        greenways_df[street] = pd.concat([greenways_df[street], section_df], ignore_index=True)

In [21]:
# aggregate
for street, df in greenways_df.items():
    display(Markdown(f'## {street}'))
    print(f'Total collisions {len(df)}\n')
    display(Markdown(f'**Total Collisions: Killed or Injured**'))
    display(df.agg({'NoKilled': ['sum'], 'NoInjured' : ['sum']}))
    print(f'Collisions with injuries: %s' %len(df.loc[(df['NoInjured'] > 0)]))
    print(f'Collisions with deaths: %s' %len(df.loc[(df['NoKilled'] > 0)]))

    bikes_df = df.loc[(df['InvWith'] == 'Bicycle') | (df['VehType1'] == 'Bicycle') | (df['VehType2'] == 'Bicycle')]
    display(Markdown(f'**Collisions Involving Bikes: Killed or Injured**'))
    display(bikes_df.agg({'NoKilled': ['sum'], 'NoInjured' : ['sum']}))
    print(f'Collisions with injuries: %s' %len(bikes_df.loc[(df['NoInjured'] > 0)]))
    print(f'Collisions with deaths: %s' %len(bikes_df.loc[(df['NoKilled'] > 0)]))

    peds_df = df.loc[(df['InvWith'] == 'Pedestrian') | (df['VehType1'] == 'Pedestrian') | (df['VehType2'] == 'Pedestrian')]
    display(Markdown(f'**Collisions Involving Pedestrians: Killed or Injured**'))
    display(peds_df.agg({'NoKilled': ['sum'], 'NoInjured' : ['sum']}))
    print(f'Collisions with injuries: %s' %len(peds_df.loc[(df['NoInjured'] > 0)]))
    print(f'Collisions with deaths: %s' %len(peds_df.loc[(df['NoKilled'] > 0)]))

    child_df = df.loc[(df['AgeGroup1'] == 'Child') | (df['AgeGroup2'] == 'Child')]
    display(Markdown(f'**Collisions Involving Children: Killed or Injured**'))
    display(child_df.agg({'NoKilled': ['sum'], 'NoInjured' : ['sum']}))
    print(f'Collisions with injuries: %s' %len(child_df.loc[(df['NoInjured'] > 0)]))
    print(f'Collisions with deaths: %s' %len(child_df.loc[(df['NoKilled'] > 0)]))

    senior_df = df.loc[(df['AgeGroup1'] == 'Senior') | (df['AgeGroup2'] == 'Senior')]
    display(Markdown(f'**Collisions Involving Seniors: Killed or Injured**'))
    display(senior_df.agg({'NoKilled': ['sum'], 'NoInjured' : ['sum']}))
    print(f'Collisions with injuries: %s' %len(senior_df.loc[(df['NoInjured'] > 0)]))
    print(f'Collisions with deaths: %s' %len(senior_df.loc[(df['NoKilled'] > 0)]))
    
    # plot
    fig = px.density_mapbox(df, lat='lat', lon='lon', radius=4,
                            center=dict(lat=34.15127, lon=-118.1369), zoom=13,
                            mapbox_style="stamen-terrain", height=1000, )
    # fig.show()
    
    # Cause
    df.Cause.value_counts()

## El Molino

Total collisions 489



**Total Collisions: Killed or Injured**

,NoKilled,NoInjured
sum,2,289


Collisions with injuries: 203
Collisions with deaths: 2


...

,NoKilled,NoInjured
sum,0,17


Collisions with injuries: 14
Collisions with deaths: 0


**Collisions Involving Seniors: Killed or Injured**

,NoKilled,NoInjured
sum,0,20


Collisions with injuries: 15
Collisions with deaths: 0
